## Deep Learning

In [15]:
import os
import pandas as pd
import math
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns  #Seaborn is another powerful visulization library for Python
from sklearn.linear_model import Ridge
import os

In [16]:
path = "C:\\Users\jmatney\Documents\GitHub\CRRA\data\hybrid"
df = pd.read_csv(os.path.join(path, "CRRA_data.csv"))

In [17]:
df.head(5)

,x,y,Ind_DEM_m,Ind_DEM_Std_ft,Ind_VOFD_ft,Ind_HOFD_ft,Ind_Point_Drainage_Area_sqmi,Ind_Catchment_Drainage_Area_sqmi,Ind_Slope_deg,Ind_Wetness_index,...,Dep_500yr_Depth_ft,Other_Catchment_ID,Other_HasBLE,Other_Soils_Mukey,Ind_Soil_Drainage_Class_Typical_index,Ind_Soil_Drainage_Class_Wettest_index,Ind_Soil_Type_index,Ind_Soil_Flood_Freq_Typical_index,Ind_Soil_Flood_Freq_Wettest_index,Ind_Soil_Hydric_Soils_percent
0,224898.134159,4.715585e+06,558.8,0.219963,82.899660,2412.499300,0.000000,1.110116,1.012812,8.640562,...,0.000000,250,1,295347,6,6,7,5,5,95
1,225928.134159,4.719415e+06,485.8,1.815682,0.099976,72.426410,0.000000,14.192085,3.754432,7.329027,...,0.386475,101,1,295357,7,7,4,5,5,95
2,226568.134159,4.725335e+06,508.7,0.369994,0.200012,24.142136,0.000309,2.846564,1.406812,10.509094,...,0.000000,7,1,295438,7,7,4,2,2,95
3,228068.134159,4.723095e+06,537.7,1.561457,5.500000,264.852780,0.000000,0.802780,6.989694,6.703982,...,0.000000,21,1,295307,3,3,5,5,5,0
4,219088.134159,4.719115e+06,545.2,0.360797,73.099550,4091.616500,0.000000,0.763166,1.505312,8.244166,...,0.000000,160,1,295338,5,5,7,5,5,5


In [21]:
df_data = df.loc[:, df.columns != 'x']
df_data = df_data.loc[:, df_data.columns != 'y']

In [22]:
df_data.columns

Index(['Ind_DEM_m', 'Ind_DEM_Std_ft', 'Ind_VOFD_ft', 'Ind_HOFD_ft',
       'Ind_Point_Drainage_Area_sqmi', 'Ind_Catchment_Drainage_Area_sqmi',
       'Ind_Slope_deg', 'Ind_Wetness_index', 'Ind_Stream_Lakes_boolean',
       'Ind_NYS_Stream_boolean', 'Ind_Stream_Order_index',
       'Ind_Land_Use_class', 'Ind_Annual_Rainfall_mm', 'Dep_100yr_Depth_ft',
       'Dep_100yr_10pct_Depth_ft', 'Dep_100yr_20pct_Depth_ft',
       'Dep_100yr_plus_Depth_ft', 'Dep_500yr_Depth_ft', 'Other_Catchment_ID',
       'Other_HasBLE', 'Other_Soils_Mukey',
       'Ind_Soil_Drainage_Class_Typical_index',
       'Ind_Soil_Drainage_Class_Wettest_index', 'Ind_Soil_Type_index',
       'Ind_Soil_Flood_Freq_Typical_index',
       'Ind_Soil_Flood_Freq_Wettest_index', 'Ind_Soil_Hydric_Soils_percent'],
      dtype='object')

In [23]:
#Define a variable for each type of feature
target = ["Dep_100yr_Depth_ft"]
columns = [x for x in df if x != "Dep_100yr_Depth_ft"]
# numeric_columns = [x for x in columns if x != "subwatershed"]

In [24]:
# Normalise
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_data)

In [25]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
# Fixed dimensions
input_dim = df_data.shape[1]  # 8
encoding_dim = 3
# Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(6, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(4, activation="tanh")(encoder_layer_1)
encoder_layer_3 = Dense(encoding_dim, activation="tanh")(encoder_layer_2)

In [ ]:
# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_3)
# Use the model to predict the factors which sum up the information of interest rates.
encoded_data = pd.DataFrame(encoder.predict(data_scaled))
encoded_data.columns = ['factor_1', 'factor_2', 'factor_3']

In [ ]:
encoded_data

## Min Max Scaler

In [ ]:
X = df_data.loc[:, df_data.columns != 'Dep_100yr_Depth_ft']
y = df_data.loc[:, df_data.columns == 'Dep_100yr_Depth_ft']

In [10]:
scaler = MinMaxScaler()

# create target scaler object
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y)

MinMaxScaler()

## Train Test Split

In [11]:
#Create train and test dataset with an 80:20 split
x_train, x_test, y_train, y_test = train_test_split(df_data[numeric_columns],df_data[target],test_size=0.2,random_state=2018)
# Further divide training dataset into train and validation dataset with an 90:10 split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=2018)

# transform target variables
y_train = scalarY.transform(y_train)
y_test = scalarY.transform(y_test)
y_val = scalarY.transform(y_val)

x_train = scalarX.transform(x_train)
x_test = scalarX.transform(x_test)
x_val = scalarX.transform(x_val)

#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (72, 40)
Shape of x_val: (8, 40)
Shape of x_test: (20, 40)
Shape of y_train: (72, 1)
Shape of y_val: (8, 1)
Shape of y_test: (20, 1)


In [12]:

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=4)]

model4 = Sequential()
model4.add(Dense(350,input_dim = 40,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(40,)))
model4.add(Dense(1,activation = "linear"))

model4.compile(optimizer='adam',loss="mean_squared_error",
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

model4.fit(x_train,y_train, 
           validation_data=(x_val,y_val), 
           epochs=450,
           batch_size=64)
#,callbacks=callbacks)

result = model4.evaluate(x_test,y_test)

for i in range(len(model4.metrics_names)):
    print("Metric ",model4.metrics_names[i],":",str(round(result[i],2)))
    


Train on 72 samples, validate on 8 samples
Epoch 1/450
72/72 [==============================] - 1s 8ms/step - loss: 0.1008 - root_mean_squared_error: 0.3251 - val_loss: 0.0551 - val_root_mean_squared_error: 0.3101
Epoch 2/450
72/72 [==============================] - 0s 445us/step - loss: 0.0841 - root_mean_squared_error: 0.3035 - val_loss: 0.0116 - val_root_mean_squared_error: 0.2941
Epoch 3/450
72/72 [==============================] - 0s 361us/step - loss: 0.0429 - root_mean_squared_error: 0.2701 - val_loss: 0.0167 - val_root_mean_squared_error: 0.2667
Epoch 4/450
72/72 [==============================] - 0s 417us/step - loss: 0.0460 - root_mean_squared_error: 0.2564 - val_loss: 0.0125 - val_root_mean_squared_error: 0.2530
Epoch 5/450
72/72 [==============================] - 0s 361us/step - loss: 0.0397 - root_mean_squared_error: 0.2450 - val_loss: 0.0116 - val_root_mean_squared_error: 0.2420
Epoch 6/450
72/72 [==============================] - 0s 375us/step - loss: 0.0342 - root_mean_

Epoch 94/450
72/72 [==============================] - 0s 292us/step - loss: 0.0050 - root_mean_squared_error: 0.1370 - val_loss: 0.0255 - val_root_mean_squared_error: 0.1370
Epoch 95/450
72/72 [==============================] - 0s 306us/step - loss: 0.0050 - root_mean_squared_error: 0.1366 - val_loss: 0.0268 - val_root_mean_squared_error: 0.1366
Epoch 96/450
72/72 [==============================] - 0s 417us/step - loss: 0.0066 - root_mean_squared_error: 0.1362 - val_loss: 0.0242 - val_root_mean_squared_error: 0.1362
Epoch 97/450
72/72 [==============================] - 0s 278us/step - loss: 0.0057 - root_mean_squared_error: 0.1358 - val_loss: 0.0201 - val_root_mean_squared_error: 0.1358
Epoch 98/450
72/72 [==============================] - 0s 250us/step - loss: 0.0056 - root_mean_squared_error: 0.1353 - val_loss: 0.0188 - val_root_mean_squared_error: 0.1353
Epoch 99/450
72/72 [==============================] - 0s 292us/step - loss: 0.0045 - root_mean_squared_error: 0.1349 - val_loss: 0

Epoch 141/450
72/72 [==============================] - 0s 278us/step - loss: 0.0034 - root_mean_squared_error: 0.1211 - val_loss: 0.0233 - val_root_mean_squared_error: 0.1211
Epoch 142/450
72/72 [==============================] - 0s 278us/step - loss: 0.0038 - root_mean_squared_error: 0.1209 - val_loss: 0.0281 - val_root_mean_squared_error: 0.1209
Epoch 143/450
72/72 [==============================] - 0s 278us/step - loss: 0.0025 - root_mean_squared_error: 0.1206 - val_loss: 0.0291 - val_root_mean_squared_error: 0.1206
Epoch 144/450
72/72 [==============================] - 0s 306us/step - loss: 0.0041 - root_mean_squared_error: 0.1204 - val_loss: 0.0248 - val_root_mean_squared_error: 0.1204
Epoch 145/450
72/72 [==============================] - 0s 361us/step - loss: 0.0033 - root_mean_squared_error: 0.1201 - val_loss: 0.0240 - val_root_mean_squared_error: 0.1201
Epoch 146/450
72/72 [==============================] - 0s 363us/step - loss: 0.0030 - root_mean_squared_error: 0.1198 - val_l

Epoch 188/450
72/72 [==============================] - 0s 319us/step - loss: 0.0054 - root_mean_squared_error: 0.1111 - val_loss: 0.0108 - val_root_mean_squared_error: 0.1111
Epoch 189/450
72/72 [==============================] - 0s 306us/step - loss: 0.0042 - root_mean_squared_error: 0.1109 - val_loss: 0.0113 - val_root_mean_squared_error: 0.1109
Epoch 190/450
72/72 [==============================] - 0s 320us/step - loss: 0.0023 - root_mean_squared_error: 0.1107 - val_loss: 0.0123 - val_root_mean_squared_error: 0.1107
Epoch 191/450
72/72 [==============================] - 0s 306us/step - loss: 0.0024 - root_mean_squared_error: 0.1105 - val_loss: 0.0157 - val_root_mean_squared_error: 0.1105
Epoch 192/450
72/72 [==============================] - 0s 306us/step - loss: 0.0040 - root_mean_squared_error: 0.1103 - val_loss: 0.0182 - val_root_mean_squared_error: 0.1103
Epoch 193/450
72/72 [==============================] - 0s 347us/step - loss: 0.0045 - root_mean_squared_error: 0.1102 - val_l

Epoch 235/450
72/72 [==============================] - 0s 361us/step - loss: 0.0022 - root_mean_squared_error: 0.1040 - val_loss: 0.0145 - val_root_mean_squared_error: 0.1040
Epoch 236/450
72/72 [==============================] - 0s 347us/step - loss: 0.0027 - root_mean_squared_error: 0.1039 - val_loss: 0.0150 - val_root_mean_squared_error: 0.1039
Epoch 237/450
72/72 [==============================] - 0s 335us/step - loss: 0.0027 - root_mean_squared_error: 0.1037 - val_loss: 0.0160 - val_root_mean_squared_error: 0.1037
Epoch 238/450
72/72 [==============================] - 0s 285us/step - loss: 0.0023 - root_mean_squared_error: 0.1036 - val_loss: 0.0181 - val_root_mean_squared_error: 0.1036
Epoch 239/450
72/72 [==============================] - 0s 292us/step - loss: 0.0017 - root_mean_squared_error: 0.1034 - val_loss: 0.0202 - val_root_mean_squared_error: 0.1034
Epoch 240/450
72/72 [==============================] - 0s 264us/step - loss: 0.0018 - root_mean_squared_error: 0.1033 - val_l

72/72 [==============================] - 0s 278us/step - loss: 0.0022 - root_mean_squared_error: 0.0942 - val_loss: 0.0297 - val_root_mean_squared_error: 0.0942
Epoch 328/450
72/72 [==============================] - 0s 296us/step - loss: 0.0015 - root_mean_squared_error: 0.0941 - val_loss: 0.0338 - val_root_mean_squared_error: 0.0941
Epoch 329/450
72/72 [==============================] - 0s 319us/step - loss: 0.0019 - root_mean_squared_error: 0.0940 - val_loss: 0.0316 - val_root_mean_squared_error: 0.0941
Epoch 330/450
72/72 [==============================] - 0s 320us/step - loss: 0.0015 - root_mean_squared_error: 0.0940 - val_loss: 0.0268 - val_root_mean_squared_error: 0.0940
Epoch 331/450
72/72 [==============================] - 0s 306us/step - loss: 0.0018 - root_mean_squared_error: 0.0939 - val_loss: 0.0240 - val_root_mean_squared_error: 0.0939
Epoch 332/450
72/72 [==============================] - 0s 292us/step - loss: 0.0015 - root_mean_squared_error: 0.0938 - val_loss: 0.0234 - 

Epoch 374/450
72/72 [==============================] - 0s 389us/step - loss: 0.0014 - root_mean_squared_error: 0.0908 - val_loss: 0.0258 - val_root_mean_squared_error: 0.0908
Epoch 375/450
72/72 [==============================] - 0s 360us/step - loss: 0.0011 - root_mean_squared_error: 0.0907 - val_loss: 0.0232 - val_root_mean_squared_error: 0.0907
Epoch 376/450
72/72 [==============================] - 0s 400us/step - loss: 0.0018 - root_mean_squared_error: 0.0906 - val_loss: 0.0246 - val_root_mean_squared_error: 0.0907
Epoch 377/450
72/72 [==============================] - 0s 459us/step - loss: 0.0016 - root_mean_squared_error: 0.0906 - val_loss: 0.0274 - val_root_mean_squared_error: 0.0906
Epoch 378/450
72/72 [==============================] - 0s 305us/step - loss: 0.0020 - root_mean_squared_error: 0.0905 - val_loss: 0.0303 - val_root_mean_squared_error: 0.0905
Epoch 379/450
72/72 [==============================] - 0s 273us/step - loss: 0.0029 - root_mean_squared_error: 0.0905 - val_l

Epoch 421/450
72/72 [==============================] - 0s 361us/step - loss: 0.0015 - root_mean_squared_error: 0.0880 - val_loss: 0.0222 - val_root_mean_squared_error: 0.0880
Epoch 422/450
72/72 [==============================] - 0s 375us/step - loss: 0.0015 - root_mean_squared_error: 0.0880 - val_loss: 0.0242 - val_root_mean_squared_error: 0.0880
Epoch 423/450
72/72 [==============================] - 0s 306us/step - loss: 0.0022 - root_mean_squared_error: 0.0879 - val_loss: 0.0219 - val_root_mean_squared_error: 0.0879
Epoch 424/450
72/72 [==============================] - 0s 306us/step - loss: 0.0011 - root_mean_squared_error: 0.0879 - val_loss: 0.0190 - val_root_mean_squared_error: 0.0879
Epoch 425/450
72/72 [==============================] - 0s 348us/step - loss: 0.0018 - root_mean_squared_error: 0.0878 - val_loss: 0.0188 - val_root_mean_squared_error: 0.0878
Epoch 426/450
72/72 [==============================] - ETA: 0s - loss: 0.0015 - root_mean_squared_error: 0.087 - 0s 347us/ste